In [5]:
! pip install pyttsx3
! pip install pydub

In [9]:
import pyttsx3
import json
import tempfile
import os
from pydub import AudioSegment

# Load transcript
filename = "call_2025-05-01_15-45.json"
with open(filename, "r") as f:
    data = json.load(f)

# Initialize TTS engine
engine = pyttsx3.init()
voices = engine.getProperty('voices')

# Choose voices (adjust index if needed)
agent_voice = voices[1].id  # female
customer_voice = voices[0].id  # male

# Create list to hold temporary file paths
temp_files = []

# Generate TTS for each line
for i, item in enumerate(data["transcript"]):
    with tempfile.NamedTemporaryFile(delete=False, suffix=".mp3") as tmp:
        voice = agent_voice if item["speaker"] == "agent" else customer_voice
        engine.setProperty('voice', voice)
        engine.save_to_file(item["text"], tmp.name)
        temp_files.append(tmp.name)

engine.runAndWait()

# Combine all audio chunks
combined = AudioSegment.empty()
for file in temp_files:
    combined += AudioSegment.from_file(file)

# Export final MP3
combined.export(f"{filename}.mp3", format="mp3")

# Clean up temporary files
for file in temp_files:
    os.remove(file)
